# Advanced Video Colorization with Sequence-Based Model

This notebook demonstrates video colorization using an advanced sequence-based model, conceptually similar to architectures like BiSTNet. Such models consider multiple frames at once to improve temporal consistency and reduce flickering compared to single-frame colorization methods.

## Setup

Import necessary modules and set up the environment.

In [ ]:
#NOTE:  This must be the first DeOldify cell to run! Select GPU environment in Colab for best performance.
from deoldify import device
from deoldify.device_id import DeviceId

#Choices: CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)

In [ ]:
import warnings 
from deoldify.visualize import get_advanced_video_colorizer, show_video_in_notebook
import matplotlib.pyplot as plt

# Matplotlib style for dark background, optional
plt.style.use('dark_background')

## Initialize the Advanced Video Colorizer

This step loads the sequence-based model. The `n_frames_input` parameter (defaulting to 5 in the model setup) determines how many frames the model considers simultaneously. A higher number can lead to better temporal consistency but may be more computationally intensive.

In [ ]:
# Define parameters here for clarity and easy modification
render_factor = 21 
watermarked = True 
debug_mode_enabled = True # Set to False to disable debug image saving
debug_output_dir = './debug_frames' if debug_mode_enabled else None
debug_frame_prefix_name = 'myvideo' if debug_mode_enabled else None

colorizer = get_advanced_video_colorizer(
    render_factor=render_factor, 
    debug_dir=debug_output_dir, 
    debug_frame_prefix=debug_frame_prefix_name
) 

# Example of using specific weights (if available in future models):
# colorizer = get_advanced_video_colorizer(
#     weights_name='MyAdvancedModel.pth', 
#     n_frames_input=7, # This would need to be a parameter in get_advanced_video_colorizer if configurable
#     render_factor=render_factor, 
#     debug_dir=debug_output_dir, 
#     debug_frame_prefix=debug_frame_prefix_name
# )

## Instructions

### `render_factor`
Determines the resolution at which the colorization model processes the video. Lower values render faster, and colors might appear more vibrant. Higher values can be better for high-quality input but may wash out colors slightly. Default used here is 21. Max is ~44 on 11GB GPUs.

### `watermarked`
Set to `True` to apply a watermark to the output video, `False` to disable.

### `debug_mode_enabled`, `debug_output_dir`, `debug_frame_prefix_name`
These parameters in the cell above control the debugging feature. If `debug_mode_enabled` is `True`, intermediate processing images will be saved to the `debug_output_dir` with the specified `debug_frame_prefix_name`.

### `result_path`
This will be automatically determined. The final colorized video will be in the `video/result/` directory.

**Important Note:** Please ensure you have a video file at `video/source/test_video.mp4` or change the path in the `colorize_from_file_name` call below to your local video file. Also, create the `./video/source/` directories if they don't exist before running the next cell.

## Colorize Video!

In [ ]:
# The render_factor and watermarked variables are used from the cell where the colorizer was initialized.
# You can override them here if you want a different setting for this specific run, e.g.:
# render_factor = 15
# watermarked = False

file_to_colorize = 'video/source/test_video.mp4'
print(f"Attempting to colorize local file: {file_to_colorize}")

try:
    video_path = colorizer.colorize_from_file_name(
        file_name=file_to_colorize, 
        render_factor=render_factor, # Uses render_factor defined when colorizer was created
        watermarked=watermarked    # Uses watermarked defined when colorizer was created
    )
    print(f"Colorized video saved to: {video_path}")
    show_video_in_notebook(video_path)
except FileNotFoundError:
    print(f"ERROR: File not found at {file_to_colorize}. Please check the path and ensure the video exists.")
except Exception as e:
    print(f"An error occurred during video colorization: {e}")

## Discussion & Comparison

The sequence-based model used here is designed to produce more temporally stable colorization with reduced flickering compared to models that process each frame independently. 

Consider colorizing the same video using the original `VideoColorizer.ipynb` (which uses a single-frame model) and compare the results. Pay attention to:
- Flickering in areas that should have consistent color.
- Color consistency of moving objects.

Note: The actual visual improvement depends heavily on the underlying model architecture and training. This notebook provides the *framework* for using such advanced models.

## Frame Preview (Optional)

The `plot_transformed_image` function visualizes how a single frame is colorized. When used with the `VideoColorizerFilter`, this shows the result of its single-image processing mode (which duplicates the frame to form a sequence). This preview might not fully represent the temporal benefits seen in the full video but can be useful for checking general color quality for a given `render_factor`.

In [ ]:
# This uses the .P() method of the filter, which for VideoColorizerFilter duplicates the frame.
# Ensure you have processed a video first so that 'bwframes' are available, or provide a direct image path.

print("Frame preview for sequence models shows single-frame duplication behavior.")
print("To test frame previews, ensure 'video/bwframes/test_video/00001.jpg' exists from a previous run, or point to a valid image.")
# Example, if 'test_video.mp4' was processed and file_to_colorize was 'video/source/test_video.mp4':
# import os
# from pathlib import Path # Ensure Path is imported if used here
# video_name_stem = Path(file_to_colorize).stem
# frame_preview_path = 'video/bwframes/' + video_name_stem + '/00001.jpg' 
# if os.path.exists(frame_preview_path):
#   colorizer.vis.plot_transformed_image(frame_preview_path, render_factor=render_factor, display_render_factor=True, figsize=(8,8))
# else:
#   print(f"Preview frame {frame_preview_path} not found. Run video colorization first or provide a direct image path.")